In [ ]:
!pip install -r requirements.txt

In [1]:
req = !pip freeze 

In [2]:
excluir = ['pywin32']
with open('requirements.txt', 'w') as f:
    for linha in req:
        if not any(pacote in linha for pacote in excluir):
            f.write(linha+'\n')

### Bibliotecas

In [248]:
%%writefile "F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\services\at_generic_methods.py"
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import time
from statsbombpy import sb
from mplsoccer.pitch import Pitch
from SPARQLWrapper import SPARQLWrapper, JSON


@st.cache_data
def get_sb_competitions():
    '''
    Função para obter as competições disponíveis no StatsBomb
    '''
    df = sb.competitions()
    return df

@st.cache_data
def get_sb_matches(competition_id, season_id):
    '''
    Função para obter as partidas de uma competição e temporada específicas
    '''
    df = sb.matches(competition_id=competition_id, season_id=season_id)
    return df

@st.cache_data
def get_sb_events(match_id):
    '''
    Função para obter os eventos de uma partida específica
    '''
    df = sb.events(match_id=match_id)
    return df

@st.cache_data
def get_sb_events_types(match_id):
    '''
    Função para obter os eventos de uma partida específica de um tipo específico (ex: passes, chutes, etc.)
    '''
    df = sb.events(match_id=match_id, split=True, flatten_attrs=False)
    return df

@st.cache_data
def get_sb_events_type(match_id, event_type):
    '''
    Função para obter os eventos de uma partida específica de um tipo específico (ex: passes, chutes, etc.)
    '''
    df = sb.events(match_id=match_id, split=True, flatten_attrs=False)[event_type]
    return df

def select_match():
    '''
    Função para selecionar a partida no sidebar do Streamlit e armazenar as informações da competição, temporada e partida selecionadas em st.session_state para uso posterior na aplicação
    '''
    df_comp = get_sb_competitions()
    competicoes = df_comp[['competition_name','competition_id']].drop_duplicates()
    competicoes.reset_index(drop=True, inplace=True)
    with st.sidebar:
        st.markdown("## Selecionar partida")
        try:
            competicao_selecionada = st.selectbox('Selecione a competição', competicoes['competition_name'], index=int(competicoes['competition_name'].loc[competicoes['competition_name'] == st.session_state['competicao_nome']].index[0]))
        except:
            competicao_selecionada = st.selectbox('Selecione a competição', competicoes['competition_name'])
        competicao_id = competicoes[competicoes['competition_name'] == competicao_selecionada]['competition_id'].values[0]

        if competicao_selecionada:
            temporadas = df_comp[df_comp['competition_name'] == competicao_selecionada][['season_name', 'season_id']].drop_duplicates().reset_index(drop=True)
            try:
                temporada_selecionada = st.selectbox('Selecione a temporada', temporadas['season_name'], index=int(temporadas[temporadas['season_name'] == st.session_state['temporada_nome']].index[0]))
            except:
                temporada_selecionada = st.selectbox('Selecione a temporada', temporadas['season_name'])
            temporada_id = temporadas[temporadas['season_name'] == temporada_selecionada]['season_id'].values[0]

        if competicao_id and temporada_id:
            partidas = get_sb_matches(competition_id=competicao_id, season_id=temporada_id)
            partidas['partida'] = partidas['home_team'] + ' x ' + partidas['away_team'] + ' - ' + partidas['match_date']

            try:
                partida_selecionada = st.selectbox('Selecione a partida', partidas['partida'], index=int(partidas[partidas['partida'] == st.session_state['partida_nome']].index[0]))
            except:
                partida_selecionada = st.selectbox('Selecione a partida', partidas['partida'])
            partida_id = partidas[partidas['partida'] == partida_selecionada]['match_id'].values[0]

        if partida_id:
            cols = st.columns([0.5,0.5])
            team_a = partidas[partidas['match_id'] == partida_id]['home_team'].values[0]
            team_b = partidas[partidas['match_id'] == partida_id]['away_team'].values[0]
            with cols[0]:
                try:
                    st.session_state['team_a_color'] = st.color_picker(f'Cor do time: {team_a}', st.session_state['team_a_color'])
                except:
                    st.session_state['team_a_color'] = st.color_picker(f'Cor do time: {team_a}', '#111111')
            with cols[1]:
                try:
                    st.session_state['team_b_color'] = st.color_picker(f'Cor do time: {team_b}', st.session_state['team_b_color'])
                except:
                    st.session_state['team_b_color'] = st.color_picker(f'Cor do time: {team_b}', '#000000')

        st.session_state['competicao_nome'] = competicao_selecionada
        st.session_state['competicao_id'] = competicao_id
        st.session_state['temporada_nome'] = temporada_selecionada
        st.session_state['temporada_id'] = temporada_id
        st.session_state['partida_nome'] = partida_selecionada
        st.session_state['partida_id'] = partida_id
        st.session_state['team_a'] = team_a
        st.session_state['team_b'] = team_b

def obter_foto_jogador(nome_jogador):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query = f"""
    SELECT ?thumbnail WHERE {{
        ?player a dbo:SoccerPlayer ;
                foaf:name "{nome_jogador}"@en ;
                dbo:thumbnail ?thumbnail .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if results["results"]["bindings"]:
        foto_url = results["results"]["bindings"][0]["thumbnail"]["value"]
        return foto_url
    else:
        return None

Overwriting F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\services\at_generic_methods.py


### App

In [211]:
%%writefile "F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\app.py"
import streamlit as st


intro_page = st.Page("Intro.py", title="Introdução", icon="📑")
partida = st.Page("Partida.py", title="Dados da partida", icon="1️⃣")
jogadores = st.Page("Jogadores.py", title="Comparar jogadores", icon="2️⃣")
pg = st.navigation([intro_page, partida, jogadores])

st.set_page_config(
        page_title="Intro",
        page_icon="Infnet_logo.png",
        layout="wide",
        initial_sidebar_state = "expanded")

pg.run()

Overwriting F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\app.py


### Capa

In [209]:
%%writefile "F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\Intro.py"
import streamlit as st

html_p = """<p style='text-align: center; font-size:%spx;'><b>%s</b></p>"""

github_link = '''https://github.com/Leonidas-Vitor/AT---Desenvolvimento-Front-End-com-Python'''
email = '''leonidas.almeida@al.infnet.edu.br'''

#st.divider()
columns = st.columns([0.3,0.7])
with columns[0]:
    st.image('Infnet_logo.png',width=200)

    #st.markdown(html_p % tuple([25,'Desenvolvimento Front-End com Python']),unsafe_allow_html = True)
    #st.markdown(html_p % tuple([25,'TP1']),unsafe_allow_html = True)

with columns[1]:
    st.markdown('''<h1 style='text-align: center; '><b>INSTITUTO INFNET</b></h1>''',unsafe_allow_html = True)
    st.markdown(html_p % tuple([35,"ESCOLA SUPERIOR DE TECNOLOGIA"]),unsafe_allow_html=True)
st.divider()
st.markdown(html_p % tuple([35,"AT - Desenvolvimento Front-End com Python"]),unsafe_allow_html=True)
st.markdown(html_p % tuple([25,'Aluno: Leônidas Almeida']),unsafe_allow_html = True)
st.markdown(html_p % tuple([25,f'E-mail: <a href= mailto:{email}>{email}</a>']),unsafe_allow_html = True)
st.markdown(html_p % tuple([25,f'GitHub: <a href={github_link}>Link para o repositório</a>']),unsafe_allow_html = True)



Overwriting F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\Intro.py


### Partida

In [410]:
%%writefile "F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\Partida.py"
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import time
from statsbombpy import sb
from mplsoccer.pitch import Pitch
from services import at_generic_methods as at_g

at_g.select_match()

st.title("AT - Desenvolvimento Front-End com Python")
st.subheader("Estatísticas básicas da partida", divider=True)

if (st.session_state['competicao_nome'] and st.session_state['temporada_nome'] and st.session_state['partida_nome']):
    match_events = at_g.get_sb_events(match_id=st.session_state['partida_id'])
    st.success("Partida selecionada com sucesso!")
    cols = st.columns([0.3,0.2,0.2,0.2,0.2])
    with cols[0]:
        with st.expander("Informações da partida selecionada", expanded=True):
            st.write("Competição: " + st.session_state['competicao_nome'])
            st.write("Temporada: " + st.session_state['temporada_nome'])
            st.write("Partida: " + st.session_state['partida_nome'])
    with cols[1]:
        df_comp = at_g.get_sb_competitions()
        st.metric("Placar", at_g.get_sb_matches(competition_id=st.session_state['competicao_id'], 
            season_id=st.session_state['temporada_id'])[(at_g.get_sb_matches(competition_id=st.session_state['competicao_id'], 
            season_id=st.session_state['temporada_id'])['match_id'] == st.session_state['partida_id'])][['home_score','away_score']].values[0][0].astype(str) + 
            ' x ' + at_g.get_sb_matches(competition_id=st.session_state['competicao_id'], 
            season_id=st.session_state['temporada_id'])[(at_g.get_sb_matches(competition_id=st.session_state['competicao_id'], 
            season_id=st.session_state['temporada_id'])['match_id'] == st.session_state['partida_id'])][['home_score','away_score']].values[0][1].astype(str))
            
        st.metric("Total de dribles", match_events[match_events['type'] == 'Dribble'].shape[0])
    with cols[2]:
        st.metric("Total de interceptações", match_events[match_events['type'] == 'Interception'].shape[0])
        st.metric("Total de passes", match_events[match_events['type'] == 'Pass'].shape[0])
    with cols[3]:
        st.metric("Total de faltas", match_events[match_events['type'] == 'Foul Committed'].shape[0])
        st.metric("Total de substituições", match_events[match_events['type'] == 'Substitution'].shape[0])
    with cols[4]:
        st.metric("Total de chutes", match_events[match_events['type'] == 'Shot'].shape[0])
        st.metric("Total de paralisações", match_events[match_events['type'] == 'Injury Stoppage'].shape[0])

    events = at_g.get_sb_events_types(st.session_state['partida_id'])
    keys = list(events.keys())
    event_type = st.selectbox('Selecione o tipo de evento para visualizá-los', keys)
    st.dataframe(events[event_type], use_container_width = True)
else :
    st.error("Selecione uma partida")
#st.dataframe(match_events[match_events['type'] == 'Injury Stoppage'])

st.subheader("Visualizações", divider=True)

#@st.cache_data
def plot_passes():
    with st.container():
        passes = at_g.get_sb_events_type(match_id=st.session_state['partida_id'], event_type='passes')

        #st.write(passes)

        team_a = st.session_state['team_a']
        team_b = st.session_state['team_b']

        passes_team_a = passes[passes['possession_team'] == team_a][['possession_team','team','location','pass']]
        passes_team_a['end_location'] = passes_team_a['pass'].apply(lambda x: x['end_location'])
        passes_team_a['sx'] = passes_team_a['location'].apply(lambda x: x[0])
        passes_team_a['sy'] = passes_team_a['location'].apply(lambda x: x[1])
        passes_team_a['ex'] = passes_team_a['end_location'].apply(lambda x: x[0])
        passes_team_a['ey'] = passes_team_a['end_location'].apply(lambda x: x[1])
        passes_team_a.drop(['pass','end_location','location'], axis=1, inplace=True)

        passes_team_b = passes[passes['possession_team'] == team_b][['possession_team','team','location','pass']]
        passes_team_b['end_location'] = passes_team_b['pass'].apply(lambda x: x['end_location'])
        passes_team_b['sx'] = passes_team_b['location'].apply(lambda x: x[0])
        passes_team_b['sy'] = passes_team_b['location'].apply(lambda x: x[1])
        passes_team_b['ex'] = passes_team_b['end_location'].apply(lambda x: x[0])
        passes_team_b['ey'] = passes_team_b['end_location'].apply(lambda x: x[1])
        passes_team_b.drop(['pass','end_location','location'], axis=1, inplace=True)

        cols = st.columns([0.7,0.3])
        with cols[0]:

            colsShow = st.columns([0.5,0.5])
            with colsShow[0]:
                show_team_a = st.toggle(f'Mostrar passes {team_a}', True)
                show_team_a_errados = st.toggle(f'Mostrar passes errados {team_a}', True)
            with colsShow[1]:
                show_team_b = st.toggle(f'Mostrar passes {team_b}', True)
                show_team_b_errados = st.toggle(f'Mostrar passes errados {team_b}', True)

            st.write('')

            pitch = Pitch(pitch_type='statsbomb',pitch_color='#22312b', line_color='#c7d5cc')
            fig, ax = pitch.draw()

            if (show_team_a):
                pitch.arrows(passes_team_a['sx'], passes_team_a['sy'], passes_team_a['ex'], 
                    passes_team_a['ey'], ax=ax, color=st.session_state['team_a_color'], zorder=1, width=1, headwidth=2, headlength=2, label=team_a)
            if (show_team_a_errados):
                passes_errados_tema_a = passes_team_a[passes_team_a['team'] != passes_team_a['possession_team']]
                pitch.arrows(passes_errados_tema_a['sx'], passes_errados_tema_a['sy'], passes_errados_tema_a['ex'], 
                    passes_errados_tema_a['ey'], ax=ax, color=st.session_state['team_b_color'], zorder=1, width=1, headwidth=2, headlength=2, label=team_a + ' errados')

            if (show_team_b):
                pitch.arrows(passes_team_b['sx'], passes_team_b['sy'], passes_team_b['ex'], 
                    passes_team_b['ey'], ax=ax, color=st.session_state['team_b_color'], zorder=1, width=1, headwidth=4, headlength=4, label=team_b)
            if (show_team_b_errados):
                passes_errados_tema_b = passes_team_b[passes_team_b['team'] != passes_team_b['possession_team']]
                pitch.arrows(passes_errados_tema_b['sx'], passes_errados_tema_b['sy'], passes_errados_tema_b['ex'], 
                    passes_errados_tema_b['ey'], ax=ax, color=st.session_state['team_a_color'], zorder=1, width=1, headwidth=4, headlength=4, label=team_b + ' errados')

            ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.0), fontsize=5, facecolor='white', edgecolor='none')
            fig.set_facecolor('#808080')
            st.pyplot(fig)

        with cols[1]:
            
            passes_errados_tema_a = passes_team_a[passes_team_a['team'] != passes_team_a['possession_team']]
            passes_errados_tema_b = passes_team_b[passes_team_b['team'] != passes_team_b['possession_team']]

            colsPassesErrados = st.columns([0.5,0.5])
            with colsPassesErrados[0]:
                st.metric(f"Passes errados {team_a}", f'{round(passes_errados_tema_a.shape[0]/passes_team_a.shape[0] * 100,1)}%')
            with colsPassesErrados[1]:
                st.metric(f"Passes errados {team_b}", f'{round(passes_errados_tema_b.shape[0]/passes_team_b.shape[0] * 100,1)}%')


            fig, ax = plt.subplots(figsize=(5,8.6))
            sns.barplot(x=[team_a, team_b], y=[passes_team_a.shape[0], passes_team_b.shape[0]], palette=[st.session_state['team_a_color'], st.session_state['team_b_color']])
            #sns.countplot(x='sx', data=passes_team_a, color=st.session_state['team_a_color'], ax=ax)
            #sns.countplot(x='sx', data=passes_team_b, color=st.session_state['team_b_color'], ax=ax)
            #bar label
            ax.bar_label(ax.containers[0],fontsize=10)
            ax.bar_label(ax.containers[1],fontsize=10)
            ax.set_title('Quantidade de passes', fontsize=15)
            ax.set_facecolor('none')
            #ax.tick_params(axis='x', colors='white')
            fig.set_facecolor('#808080')
            st.pyplot(fig)

def plot_chutes():
    with st.container():
        chutes = at_g.get_sb_events_type(match_id=st.session_state['partida_id'], event_type='shots')

        team_a = st.session_state['team_a']
        team_b = st.session_state['team_b']

        chutes_team_a = chutes[chutes['possession_team'] == team_a][['location','shot']]
        chutes_team_a['end_location'] = chutes_team_a['shot'].apply(lambda x: x['end_location'])
        chutes_team_a['sx'] = chutes_team_a['location'].apply(lambda x: x[0])
        chutes_team_a['sy'] = chutes_team_a['location'].apply(lambda x: x[1])
        chutes_team_a['ex'] = chutes_team_a['end_location'].apply(lambda x: x[0])
        chutes_team_a['ey'] = chutes_team_a['end_location'].apply(lambda x: x[1])
        chutes_team_a['outcome'] = chutes_team_a['shot'].apply(lambda x: x['outcome']['name'])
        chutes_team_a.drop(['shot','end_location','location'], axis=1, inplace=True)

        chutes_team_b = chutes[chutes['possession_team'] == team_b][['location','shot']]
        chutes_team_b['end_location'] = chutes_team_b['shot'].apply(lambda x: x['end_location'])
        chutes_team_b['sx'] = chutes_team_b['location'].apply(lambda x: x[0])
        chutes_team_b['sy'] = chutes_team_b['location'].apply(lambda x: x[1])
        chutes_team_b['ex'] = chutes_team_b['end_location'].apply(lambda x: x[0])
        chutes_team_b['ey'] = chutes_team_b['end_location'].apply(lambda x: x[1])
        chutes_team_b['outcome'] = chutes_team_b['shot'].apply(lambda x: x['outcome']['name'])
        chutes_team_b.drop(['shot','end_location','location'], axis=1, inplace=True)

        cols = st.columns([0.7,0.3])
        with cols[0]:

            colsShow = st.columns([0.5,0.5])
            with colsShow[0]:
                show_team_a = st.toggle(f'Mostrar finalizações {team_a}', True)
                show_team_a_gols = st.toggle(f'Mostrar gols {team_a}', True)
            with colsShow[1]:
                show_team_b = st.toggle(f'Mostrar finalizações {team_b}', True)
                show_team_b_gols = st.toggle(f'Mostrar gols {team_b}', True)
            
            st.write('')

            pitch = Pitch(pitch_type='statsbomb',pitch_color='#22312b', line_color='#c7d5cc')
            fig, ax = pitch.draw()

            chutes = at_g.get_sb_events_type(match_id=st.session_state['partida_id'], event_type='shots')

            #st.write(chutes['shot'].str['outcome'])#['outcome'])

            if (show_team_a):
                pitch.arrows(chutes_team_a['sx'], chutes_team_a['sy'], chutes_team_a['ex'], 
                    chutes_team_a['ey'], ax=ax, color=st.session_state['team_a_color'], zorder=1, width=1, headwidth=2, headlength=2, label=team_a)
            if (show_team_a_gols):
                chutes_gols_tema_a = chutes_team_a[chutes_team_a['outcome'] == 'Goal']
                pitch.arrows(chutes_gols_tema_a['sx'], chutes_gols_tema_a['sy'], chutes_gols_tema_a['ex'], 
                    chutes_gols_tema_a['ey'], ax=ax, color=st.session_state['team_a_color'], zorder=1, width=1, headwidth=2, headlength=2, label=team_a + ' gols')

            if (show_team_b):
                pitch.arrows(chutes_team_b['sx'], chutes_team_b['sy'], chutes_team_b['ex'], 
                    chutes_team_b['ey'], ax=ax, color=st.session_state['team_b_color'], zorder=1, width=1, headwidth=4, headlength=4, label=team_b)
            if (show_team_b_gols):
                chutes_gols_tema_b = chutes_team_b[chutes_team_b['outcome'] == 'Goal']
                pitch.arrows(chutes_gols_tema_b['sx'], chutes_gols_tema_b['sy'], chutes_gols_tema_b['ex'], 
                    chutes_gols_tema_b['ey'], ax=ax, color=st.session_state['team_b_color'], zorder=1, width=1, headwidth=4, headlength=4, label=team_b + ' gols')
            
            ax.legend(loc='upper left', bbox_to_anchor=(0.0, 1.0), fontsize=5, facecolor='white', edgecolor='none')
            fig.set_facecolor('#808080')
            st.pyplot(fig)

        with cols[1]:
            gols_a = chutes_team_a[chutes_team_a['outcome'] == 'Goal'].shape[0]
            gols_b = chutes_team_b[chutes_team_b['outcome'] == 'Goal'].shape[0]

            colsChutesErrados = st.columns([0.5,0.5])
            with colsChutesErrados[0]:
                st.metric(f"Finalizações com gol {team_a}", f'{round(gols_a/chutes_team_a.shape[0] * 100,1)}%')
            with colsChutesErrados[1]:
                st.metric(f"Finalizações com gol {team_b}", f'{round(gols_b/chutes_team_b.shape[0] * 100,1)}%')

            fig, ax = plt.subplots(figsize=(5,8.6))
            sns.barplot(x=[team_a, team_b], y=[chutes_team_a.shape[0], chutes_team_b.shape[0]], palette=[st.session_state['team_a_color'], st.session_state['team_b_color']])
            #sns.countplot(x='sx', data=passes_team_a, color=st.session_state['team_a_color'], ax=ax)
            #sns.countplot(x='sx', data=passes_team_b, color=st.session_state['team_b_color'], ax=ax)
            #bar label
            ax.bar_label(ax.containers[0],fontsize=10)
            ax.bar_label(ax.containers[1],fontsize=10)
            ax.set_title('Quantidade de finalizações', fontsize=15)
            ax.set_facecolor('none')
            #ax.tick_params(axis='x', colors='white')
            fig.set_facecolor('#808080')
            st.pyplot(fig)

st.markdown("##### Passes")
plot_passes()

st.markdown("##### Finaliozações")
plot_chutes()

#------------------------------------------------------------------------------------------------------------------------------------------------------
st.markdown("##### Heatmap")

def plot_heatmap(team):
    fig, ax = plt.subplots(figsize=(10,10))
    #Contar eventos por tipo e plotar um heatmap
    events = at_g.get_sb_events_types(st.session_state['partida_id'])
    keys = list(events.keys())
    data = {}
    for key in keys:
        data[key] = events[key][events[key]['team'] == team].shape[0]
    data = pd.DataFrame(data, index=[0])
    sns.heatmap(data.corr(), annot=True, fmt='d', cmap='coolwarm', ax=ax)
    st.pyplot(fig)

#cols = st.columns([0.5,0.5])
#with cols[0]:
#    plot_heatmap(st.session_state['team_a'])
#with cols[1]:
#    plot_heatmap(st.session_state['team_b'])
#Falta relacionar eventos (passes com chutes e etc.)

#Falta mais um plot do mplsoccer

Overwriting F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\Partida.py


### Jogadores

In [397]:
%%writefile "F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\Jogadores.py"
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import time
from statsbombpy import sb
from mplsoccer.pitch import Pitch
from mplsoccer import VerticalPitch, add_image, FontManager, Sbopen
from services import at_generic_methods as at_g
from matplotlib.colors import LinearSegmentedColormap

at_g.select_match()

st.title("AT - Desenvolvimento Front-End com Python")
st.subheader("Comparar jogadores", divider=True)

metricas = st.multiselect('Selecione as métricas', ['Passes', 'Chutes', 'Dribles', 'Interceptações', 'Faltas', 'Gols', 'TaxaGol', 'EventoPressao'], 
    default=['Passes', 'Chutes', 'Dribles', 'Interceptações', 'Faltas', 'Gols','TaxaGol', 'EventoPressao'])

df = pd.DataFrame()
dic = {}
dict_b = {}

cols = st.columns(2)

with cols[0]:
    player_a_container = st.container(border = True)
with cols[1]:
    player_b_container = st.container(border = True)

def show_select_player(team):
    st.markdown(f'### {team}')
    jogadores = sb.lineups(match_id=st.session_state['partida_id'])[team][['player_name','player_id','player_nickname','jersey_number']]
    #usar nickname se tiver, senão usar nome
    jogadores['player'] = jogadores['player_nickname'].fillna(jogadores['player_name'])
    #Adicionar o número da camisa no nome
    jogadores['player_jersey'] = jogadores['player'] + ' - ' + jogadores['jersey_number'].astype(str)
    #st.write(jogadores)
    jogador = st.selectbox('Selecione o jogador', jogadores['player_jersey'],key=team)
    jogador_foto = at_g.obter_foto_jogador(jogadores[jogadores['player_jersey'] == jogador]['player'].values[0])
    if jogador_foto:
        st.markdown(
            f"""
            <div style="display: flex; justify-content: center;">
                <img src="{jogador_foto}" style="height: {250}px;">
            </div>
            """, unsafe_allow_html=True
        )
        #st.image(jogador_foto)#, use_column_width=True, output_format='auto')
    else:
        st.markdown(
            f"""
            <div style="display: flex; justify-content: center;">
                <img src="{'https://upload.wikimedia.org/wikipedia/commons/0/0a/No-image-available.png'}" style="height: {250}px;">
            </div>
            """, unsafe_allow_html=True
        )
        #st.image('https://upload.wikimedia.org/wikipedia/commons/0/0a/No-image-available.png')#, use_column_width=True, output_format='auto')
    dic[team] = {'Jogador': jogador}
    player_events = at_g.get_sb_events(match_id=st.session_state['partida_id'])
    player_events = player_events[player_events['player'] == jogadores[jogadores['player_jersey'] == jogador]['player_name'].values[0]]
    return jogador, player_events

with cols[0]:
    with player_a_container:
        jogador_a, player_a_events = show_select_player(st.session_state["team_a"])
with cols[1]:
    with player_b_container:
        jogador_b, player_b_events = show_select_player(st.session_state["team_b"])


def show_player_stats(team, player_events, player_events_compare):
    cols = st.columns(2)
    with cols[0]:
        if 'Passes' in metricas:
            passes = player_events[player_events['type'] == 'Pass'].shape[0]
            dic[team]['Passes'] = passes
            delta = passes - player_events_compare[player_events_compare['type'] == 'Pass'].shape[0]
            st.metric("Total de passes", player_events[player_events['type'] == 'Pass'].shape[0],delta)
        if 'Chutes' in metricas:
            shots = player_events[player_events['type'] == 'Shot'].shape[0]
            dic[team]['Chutes'] = shots
            delta = shots - player_events_compare[player_events_compare['type'] == 'Shot'].shape[0]
            st.metric("Total de chutes", player_events[player_events['type'] == 'Shot'].shape[0],delta)
        if 'Dribles' in metricas:
            dribles = player_events[player_events['type'] == 'Dribble'].shape[0]
            dic[team]['Dribles'] = dribles
            delta = dribles - player_events_compare[player_events_compare['type'] == 'Dribble'].shape[0]
            st.metric("Total de dribles", player_events[player_events['type'] == 'Dribble'].shape[0],delta)
        if 'Interceptações' in metricas:
            interceptacoes = player_events[player_events['type'] == 'Interception'].shape[0]
            dic[team]['Interceptações'] = interceptacoes
            delta = interceptacoes - player_events_compare[player_events_compare['type'] == 'Interception'].shape[0]
            st.metric("Total de interceptações", player_events[player_events['type'] == 'Interception'].shape[0],delta)

    with cols[1]:
        if 'Faltas' in metricas:
            faltas = player_events[player_events['type'] == 'Foul Committed'].shape[0]
            dic[team]['Faltas'] = faltas
            delta = faltas - player_events_compare[player_events_compare['type'] == 'Foul Committed'].shape[0]
            st.metric("Total de faltas", player_events[player_events['type'] == 'Foul Committed'].shape[0],delta,'inverse')

        if 'Gols' in metricas:
            gols = player_events[(player_events['type'] == 'Shot') & (player_events['shot_outcome'] == 'Goal')].shape[0]
            dic[team]['Gols'] = gols
            delta = gols - player_events_compare[(player_events_compare['type'] == 'Shot') & (player_events_compare['shot_outcome'] == 'Goal')].shape[0]
            st.metric("Total de goals", gols,delta)

        if 'TaxaGol' in metricas:
            try:
                shot_goal_rate = gols/shots * 100
            except:
                shot_goal_rate = 0
            try:
                shot_goal_rate_compare = player_events_compare[(player_events_compare['type'] == 'Shot') & (player_events_compare['shot_outcome'] == 'Goal')].shape[0]/player_events_compare[player_events_compare['type'] == 'Shot'].shape[0] * 100
            except:
                shot_goal_rate_compare = 0
            dic[team]['TaxaGol'] = shot_goal_rate
            delta = shot_goal_rate - shot_goal_rate_compare
            st.metric("Taxa de gols", f'{round(shot_goal_rate,1)}%',f'{round(delta,1)}%')
        if 'EventoPressao' in metricas:
            under_pressure = player_events[player_events['under_pressure'] == True].shape[0]
            dic[team]['EventoPressao'] = under_pressure
            delta = under_pressure - player_events_compare[player_events_compare['under_pressure'] == True].shape[0]
            st.metric("Total de evento sob pressão", under_pressure,delta)


def show_kde(player_events, color):
    st.write('### Mapa de calor')
    x = player_events['location'].apply(lambda x: x[0] if isinstance(x, (tuple, list)) else np.nan)
    y = player_events['location'].apply(lambda x: x[1] if isinstance(x, (tuple, list)) else np.nan)

    flamingo_cmap = LinearSegmentedColormap.from_list("Flamingo - 100 colors",
                                                ['#101010', color], N=100)
    pitch = VerticalPitch(line_color='#404040', line_zorder=2)
    fig, ax = pitch.draw(figsize=(4.4, 6.4))
    kde = pitch.kdeplot(x, y, ax=ax,
                        fill=True, levels=100,
                        thresh=0,
                        cut=4,
                        cmap=flamingo_cmap)
    fig.set_facecolor('none')
    ax.set_facecolor('none')
    st.pyplot(fig)


cols = st.columns(2)

with player_a_container:
    st.divider()
    show_player_stats(st.session_state["team_a"], player_a_events, player_b_events)
    show_kde(player_a_events, st.session_state['team_a_color'])

with player_b_container:
    st.divider()
    show_player_stats(st.session_state["team_b"], player_b_events, player_a_events)
    show_kde(player_b_events, st.session_state['team_b_color'])

#Download

if st.button('Download dados selecionados', key='download'):
    with st.spinner('Preparando dados...'):
        time.sleep(2)

    my_bar = st.progress(0, text='Progresso')
    for i in range(100):
        time.sleep(0.01)
        my_bar.progress(i + 1, text='Progresso')
    time.sleep(1)
    my_bar.empty()
    st.success('Download pronto para ser realizado!')
    st.download_button('Clique aqui para baixar o dataset filtrado', df.to_csv(), file_name='data.csv', mime='text/csv')


Overwriting F:\OneDrive\Graduação Ciência de dados\TPs\Desenvolvimento Front-End com Python\AT\app\Jogadores.py
